# Atmodeller examples

There are more examples in the tests/ directory which you can obtain from Atmodeller source code. Also see the Atmodeller documentation.

All code blocks require the following imports, so you must run this first.

In [ ]:
from atmodeller import (
    Species,
    InteriorAtmosphere,
    Planet,
    earth_oceans_to_hydrogen_mass,
    debug_logger,
)
from atmodeller.solubility import get_solubility_models
from atmodeller.thermodata import get_thermodata
from atmodeller.eos import get_eos_models
from atmodeller.thermodata.redox_buffers import IronWustiteBuffer
import logging
import numpy as np
import pprint

logger = debug_logger()
logger.setLevel(logging.INFO)
# For more output use DEBUG
# logger.setLevel(logging.DEBUG)

You can find the available species in Atmodeller, where the first part of the name indicates the chemical formula and the second part after the underscore indicates the phase, using `get_thermodata()`. Common phases are g for gas, l for liquid, and cr for crystalline solid.

In [ ]:
# Get thermodynamic data for species
thermodata = get_thermodata()
logger.info(thermodata.keys())

CO2_g = thermodata["CO2_g"]
# Compute the Gibbs energy relative to RT at 2000 K
gibbs_over_RT = CO2_g.get_gibbs_over_RT(2000.0)
logger.info("Gibbs over RT = %s", gibbs_over_RT)
# Compute the composition
composition = CO2_g.composition
logger.info("Composition = %s", composition)
# Etc., other methods are available to compute other quantities

Similarly, you can find the available solubility laws:

In [ ]:
solubility_models = get_solubility_models()
logger.info(solubility_models.keys())

CO2_basalt = solubility_models["CO2_basalt_dixon95"]
# Compute the concentration at fCO2=0.5 bar, 1300 K, and 1 bar
# Note that fugacity is the first argument and others are keyword only
concentration = CO2_basalt.concentration(0.5, temperature=1300, pressure=1)
logger.info("concentration = %s", concentration)

And the available real gas equations of state. Importantly, EOS that are bounded, and hence more appropriate for use in numerical computations, have a `_bounded` suffix.

In [ ]:
eos_models = get_eos_models()
logger.info(eos_models.keys())

# Get a CH4 model
CH4_eos_model = eos_models["CH4_beattie_holley58"]
# Compute the fugacity at 800 K and 100 bar
fugacity = CH4_eos_model.fugacity(800, 100)
logger.info("fugacity = %s", fugacity)
# Compute the compressibility factor at the same conditions
compressibility_factor = CH4_eos_model.compressibility_factor(800, 100)
logger.info("compressibility_factor = %s", compressibility_factor)
# Etc., other methods are available to compute other quantities

## Basic operation

Run a single model

In [ ]:
solubility_models = get_solubility_models()
# Get the available solubility models
logger.info("solubility models = %s", solubility_models.keys())

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
mass_constraints = {
    "H": h_kg,
    "O": o_kg,
}

# If you do not specify an initial solution guess then a default will be used
# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
solution = output.quick_look()

# Get complete solution as a dictionary
solution_asdict = output.asdict()
logger.info(solution_asdict)

# Write the complete solution to Excel
output.to_excel("example_single")

## Batch calculation

For a batch calculation you can provide arrays to the planet or constraints. All arrays must have the same size because for a batch calculation the array values are aligned by position.

In [ ]:
solubility_models = get_solubility_models()
# Get the available solubility models
logger.info("solubility models = %s", pprint.pformat(solubility_models.keys()))

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)

# Batch temperature and radius, where the entries correspond by position. You could also choose
# to leave one or both as scalars.
# You must specify dtype=np.float_ for surface temperature
surface_temperature = np.array([2000, 2000, 1500, 1500], dtype=np.float_)
surface_radius = 6371000.0 * np.array([1.5, 3, 1.5, 3], dtype=np.float_)

planet = Planet(surface_temperature=surface_temperature, surface_radius=surface_radius)
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
scale_factor = 5
mass_constraints = {
    # We can also batch constraints, as long as we also have a total of 4 entries
    "H": np.array([h_kg, h_kg, h_kg * scale_factor, h_kg * scale_factor], dtype=np.float_),
    "O": np.array([o_kg, o_kg * scale_factor, o_kg, o_kg * scale_factor], dtype=np.float_),
}

# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
solution = output.quick_look()

# Get complete solution as a dictionary
solution_asdict = output.asdict()
logger.info(solution_asdict)

# Write the complete solution to Excel
output.to_excel("example_batch")

## Atmodeller within a time loop (JIT compilation)

For models where you need to dynamically update constraints during the course of a time-integration, atmodeller can be utilised as follows. In essence, the model is pre-compiled using `initialise_solve` and then the compiled function is subsequently called with different arguments, which is fast. Note that the order of the arguments and the size of the arrays must be the same as those used to initialise the model, but of course the values can be different.

In [ ]:
# This first part is the initialisation stage and should appear outside of your main time loop

solubility_models = get_solubility_models()
# Get the available solubility models
logger.info("solubility models = %s", pprint.pformat(solubility_models.keys()))

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(oceans)
o_kg = 6.25774e20
mass_constraints = {
    "H": h_kg,
    "O": o_kg,
}

# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

# Precompile
interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
)

# This is the time loop, where something changes and you want to re-solve using atmodeller
for ii in range(1, 10):
    # Let's say we update the mass constraints. The number of constraints and the value size must
    # remain the same as the initialised model, but you are free to update their values. Here,
    # scale by number of earth oceans for the hydrogen mass.
    logger.info("Your code does something here to compute new masses")

    logger.info("New masses are given as inputs to Atmodeller")
    mass_constraints = {"H": h_kg * ii, "O": o_kg}
    # These solves are fast because they use the JAX-compiled code
    logger.info("Atmodeller solve using JIT compiled code")
    output = interior_atmosphere.solve(mass_constraints=mass_constraints)

    # Quick look at the solution
    solution = output.quick_look()
    logger.info("solution = %s", solution)

    # Get complete solution as a dictionary
    # This might be required to get output to feed back into other quantities during the time loop
    # solution_asdict = output.asdict()

# Monte Carlo

Exploring atmospheric compositions in a Monte Carlo model can easily be achieved with a batch 
calculation over a range of parameters. Note that in this case the same initial solution is used 
for all cases, which means the range of parameters should not be too large. If you want a larger 
range it's advisable to run several batches with different initial solutions and then stitch the 
output together.

In [ ]:
solubility_models = get_solubility_models()
# Get the available solubility models
logger.info("solubility models = %s", pprint.pformat(solubility_models.keys()))

H2_g = Species.create_gas("H2_g")
H2O_g = Species.create_gas("H2O_g", solubility=solubility_models["H2O_peridotite_sossi23"])
O2_g = Species.create_gas("O2_g")

species = (H2_g, H2O_g, O2_g)
planet = Planet()
interior_atmosphere = InteriorAtmosphere(species)

number_of_realisations = 1000
log10_number_oceans = np.random.uniform(0, 3, number_of_realisations)
number_oceans = 10**log10_number_oceans
fO2_min = -3
fO2_max = 3
fO2_log10_shifts = np.random.uniform(fO2_min, fO2_max, number_of_realisations)

oceans = 1
h_kg = earth_oceans_to_hydrogen_mass(number_oceans)
mass_constraints = {
    "H": h_kg,
}
fugacity_constraints = {O2_g.name: IronWustiteBuffer(fO2_log10_shifts)}

# Initial solution guess number density (molecules/m^3)
initial_log_number_density = 50 * np.ones(len(species), dtype=np.float_)

# Precompile
interior_atmosphere.initialise_solve(
    planet=planet,
    initial_log_number_density=initial_log_number_density,
    mass_constraints=mass_constraints,
    fugacity_constraints=fugacity_constraints,
)
output = interior_atmosphere.solve()

# Quick look at the solution
solution = output.quick_look()

# Get complete solution as a dictionary
solution_asdict = output.asdict()
logger.info(solution_asdict)

# Write the complete solution to Excel
output.to_excel("example_monte_carlo")